## Fama e Bliss (1987)

Usando os dados do Futuro DI

In [1]:
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.formula.api as sm

In [2]:
# mostra o dataframe inteiro
# pd.set_option('display.max_rows', None)

Para a base de dados, usaremos o futuro DI que funciona como uma proxy para o retorno das Letra do Tesouro Nacional (LTN). Isso porque o futuro DI é medido através da expectativa da taxa de juros livre de risco da economia, caracaterizada pelo título LTN do governo que representa um ativo livre de risco.

Os dados do futuro DI estão em periodicidade mensal, e abrangem todo o período a partir de Junho de 2012 até Outubro de 2022. Os dados foram obtidos através da Reuters cujo acesso é liberado pelo departamento de Economia da PUC-Rio.

In [3]:
futurodi = pd.read_csv('futuroDI.csv', sep=',', index_col=0)

In [4]:
futurodi

,BRPRE1M=BMF,BRPRE2M=BMF,BRPRE3M=BMF,BRPRE4M=BMF,BRPRE5M=BMF,BRPRE6M=BMF,BRPRE7M=BMF,BRPRE8M=BMF,BRPRE9M=BMF,BRPRE10M=BMF,...,BRPRE1Y=BMF,BRPRE2Y=BMF,BRPRE3Y=BMF,BRPRE4Y=BMF,BRPRE5Y=BMF,BRPRE6Y=BMF,BRPRE7Y=BMF,BRPRE8Y=BMF,BRPRE9Y=BMF,BRPRE10Y=BMF
10/31/2022,13.659,13.670,13.688,13.6990,13.6990,13.672,13.6560,13.6050,13.4820,13.3880,...,13.130,11.807,11.482,11.495,11.537,11.6190,11.6760,11.7180,11.719,11.7480
9/30/2022,13.668,13.682,13.689,13.7620,13.6540,13.688,13.6780,13.6080,13.5580,13.4800,...,13.245,11.938,11.612,11.669,11.731,11.8440,11.9130,11.9490,11.941,11.9640
8/31/2022,13.673,13.708,13.743,13.7450,13.7540,13.760,13.7490,13.7740,13.7380,13.6730,...,13.526,12.375,11.913,11.898,11.869,11.9890,12.0220,12.0650,12.060,12.0990
7/31/2022,13.513,13.619,13.724,13.7960,13.8360,13.879,13.8940,13.9150,13.9150,13.9160,...,13.823,13.064,12.714,12.737,12.863,12.8190,12.9250,12.9520,12.959,12.9800
6/30/2022,13.144,13.330,13.479,13.5930,13.7010,13.782,13.8770,13.9000,13.9230,13.9630,...,13.923,13.231,12.781,12.699,12.912,12.8150,12.9580,12.9560,12.963,12.9480
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
10/31/2012,7.096,7.099,7.089,7.0720,7.1000,7.116,7.1130,7.1230,7.1530,7.1840,...,7.252,7.749,8.149,8.465,8.636,9.0402,9.1016,9.1458,NaN,9.2285
9/30/2012,7.293,7.272,7.257,7.2440,7.2470,7.259,7.9137,8.0644,8.2187,8.3416,...,7.466,8.152,8.596,8.939,9.165,9.6752,9.7612,9.8158,NaN,9.8961
8/31/2012,7.348,7.308,7.287,7.2650,7.2560,7.259,7.7958,8.0215,8.1829,8.3180,...,7.468,8.261,8.646,8.910,9.104,9.5986,9.5960,9.6878,NaN,9.7396
7/31/2012,7.806,7.648,7.511,7.4270,7.3770,7.348,7.8232,8.0065,8.1631,8.2856,...,7.431,8.178,8.595,8.926,9.133,9.5374,9.5833,9.6812,NaN,9.8353


In [5]:
# função que torna o índice um objeto de data
futurodi.index = pd.to_datetime(futurodi.index)

In [6]:
# normalizando os dados que estavam em taxas para números decimais
futurodi = futurodi/100

In [7]:
# algumas colunas não foram utilizadas e, por isso são retiradas desde o início para despoluir a visualização do DataFrame.
futurodi = futurodi.drop(['BRPRE1M=BMF', 'BRPRE2M=BMF', 'BRPRE3M=BMF', 'BRPRE4M=BMF', 'BRPRE5M=BMF', 'BRPRE6M=BMF', 'BRPRE7M=BMF', 'BRPRE8M=BMF', 'BRPRE9M=BMF', 'BRPRE10M=BMF', 'BRPRE11M=BMF', 'BRPRE6Y=BMF', 'BRPRE7Y=BMF', 'BRPRE8Y=BMF', 'BRPRE9Y=BMF', 'BRPRE10Y=BMF'], axis=1)

In [8]:
futurodi

,BRPRE1Y=BMF,BRPRE2Y=BMF,BRPRE3Y=BMF,BRPRE4Y=BMF,BRPRE5Y=BMF
2022-10-31,0.13130,0.11807,0.11482,0.11495,0.11537
2022-09-30,0.13245,0.11938,0.11612,0.11669,0.11731
2022-08-31,0.13526,0.12375,0.11913,0.11898,0.11869
2022-07-31,0.13823,0.13064,0.12714,0.12737,0.12863
2022-06-30,0.13923,0.13231,0.12781,0.12699,0.12912
...,...,...,...,...,...
2012-10-31,0.07252,0.07749,0.08149,0.08465,0.08636
2012-09-30,0.07466,0.08152,0.08596,0.08939,0.09165
2012-08-31,0.07468,0.08261,0.08646,0.08910,0.09104
2012-07-31,0.07431,0.08178,0.08595,0.08926,0.09133


Uma breve explicação do que se trata cada coluna. Vale notar primeiro que cada coluna tem o mesmo padrão alterando apenas o número no sexto caracter. De maneira genérica, como o futuro DI é uma proxy do retorno da LTN, temos que a interpretação de $BRPRExY=BMF$ é uma proxy do retorno anualizado de uma LTN com vencimento em $t+x$ comprada em $t$.

Por exemplo, $BRPRE3Y=BMF$ é uma proxy do retorno anualizado de uma LTN com vencimento $t+3$ comprada em $t$.

Agora precisamos criar algumas taxas de retorno. No artigo Fama e Bliss (1987), definem o rendimento de um título que paga \$1 em $t+x$ comprado em $t$ como 

$$
r(x:t) = \ln\left[\frac{p(x:t+x)}{p(x:t)}\right]
$$

Como $p(x:t+x) = 1$

$$
r(x:t) = -\ln[{p(x:t)}]
$$


Para o caso brasileiro, utilizando o futuro DI como proxy, temos que $r(x:t)$ é o retorno de uma LTN comprada em $t$ que paga R\$ 1 em $t+x$. Portanto

$$
r(x:t) = (1+BRPRExY=BMF)^x - 1
$$

In [9]:
futurodi['r(1:t)'] = futurodi['BRPRE1Y=BMF']
futurodi['r(2:t)'] = ((1+futurodi['BRPRE2Y=BMF'])**2) - 1
futurodi['r(3:t)'] = ((1+futurodi['BRPRE3Y=BMF'])**3) - 1 
futurodi['r(4:t)'] = ((1+futurodi['BRPRE4Y=BMF'])**4) - 1 
futurodi['r(5:t)'] = ((1+futurodi['BRPRE5Y=BMF'])**5) - 1

Uma outra taxa que será necessária é $r(x:t+1)$, interpretada como o retorno de uma LTN comprada em $t+1$ que paga R\$1 em $t+x+1$. A equação que nos entrega essa taxa é dada por

$$
r(x:t+1) = \frac{(1+BRPRE(x+1)Y=BMF)^{x+1}}{(1+BRPRExY=BMF)} - 1
$$

In [10]:
futurodi['r(1:t+1)'] = (((1+futurodi['BRPRE2Y=BMF'])**2)/(1+futurodi['BRPRE1Y=BMF'])) -1
futurodi['r(2:t+1)'] = (((1+futurodi['BRPRE3Y=BMF'])**3)/(1+futurodi['BRPRE1Y=BMF'])) -1
futurodi['r(3:t+1)'] = (((1+futurodi['BRPRE4Y=BMF'])**4)/(1+futurodi['BRPRE1Y=BMF'])) -1
futurodi['r(4:t+1)'] = (((1+futurodi['BRPRE5Y=BMF'])**5)/(1+futurodi['BRPRE1Y=BMF'])) -1

A última taxa necessária para a construção das variáveis é $r(1:t+x)$ que representa o retorno da LTN comprada em $t+x$ que paga R\$1 em $t+x+1$ obtida através de

$$
r(1:t+x) = \frac{(1+BRPRE(x+1)Y=BMF)^{x+1}}{(1+BRPRE(x)Y=BMF)^x} -1
$$

In [11]:
futurodi['r(1:t+2)'] = (((1+futurodi['BRPRE3Y=BMF'])**3)/((1+futurodi['BRPRE2Y=BMF'])**2)) - 1
futurodi['r(1:t+3)'] = (((1+futurodi['BRPRE4Y=BMF'])**4)/((1+futurodi['BRPRE3Y=BMF'])**3)) - 1
futurodi['r(1:t+4)'] = (((1+futurodi['BRPRE5Y=BMF'])**5)/((1+futurodi['BRPRE4Y=BMF'])**4)) - 1

A taxa forward é definida como

$$
f(x,x-1:t) = r(x:t) - r(x-1:t)
$$

Enquanto a taxa de retorno no caso brasileiro, da LTN comprada em $t$ e vendida em $t+1$ que pagaria R\$1 em $t+x-1$ é dada por

$$
h(x,x-1:t+1) = \ln[p(x-1:t+1)] - \ln[p(x:t)] = r(x:t) - r(x-1:t+1)
$$

onde Fama e Bliss (1987) utilizaram $x \in \{2,3,4,5\}$

In [12]:
futurodi['f(2,1:t)'] = futurodi['r(2:t)'] - futurodi['r(1:t)']
futurodi['f(3,2:t)'] = futurodi['r(3:t)'] - futurodi['r(2:t)']
futurodi['f(4,3:t)'] = futurodi['r(4:t)'] - futurodi['r(3:t)']
futurodi['f(5,4:t)'] = futurodi['r(5:t)'] - futurodi['r(4:t)']

futurodi['h(2,1:t+1)'] = futurodi['r(2:t)'] - futurodi['r(1:t+1)']
futurodi['h(3,2:t+1)'] = futurodi['r(3:t)'] - futurodi['r(2:t+1)']
futurodi['h(4,3:t+1)'] = futurodi['r(4:t)'] - futurodi['r(3:t+1)']
futurodi['h(5,4:t+1)'] = futurodi['r(5:t)'] - futurodi['r(4:t+1)']

A regressão da Tabela 1 é dada por

$$
h(x,x-1:t+1) - r(1:t) = \alpha + \beta \left[f(x,x-1:t) - r(1:t)\right] + u_2(t+1)
$$

onde o regressor é $f(x,x-1:t) - r(1:t)$ e o regressando $h(x,x-1:t+1) - r(1:t)$

In [13]:
futurodi['y'] = futurodi['h(2,1:t+1)'] - futurodi['r(1:t)']
futurodi['x'] = futurodi['f(2,1:t)'] - futurodi['r(1:t)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $x=2$, a regressão é


$$
h(2,1:t+1) - r(1:t) = \alpha + \beta \left[f(2,1:t) - r(1:t)\right] + u_2(t+1)
$$

In [14]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.041
Model:                            OLS   Adj. R-squared:                  0.033
Method:                 Least Squares   F-statistic:                     5.194
Date:                Fri, 21 Oct 2022   Prob (F-statistic):             0.0244
Time:                        19:42:01   Log-Likelihood:                 463.51
No. Observations:                 125   AIC:                            -923.0
Df Residuals:                     123   BIC:                            -917.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0120      0.001     11.969      0.0

In [15]:
futurodi['y'] = futurodi['h(3,2:t+1)'] - futurodi['r(1:t)']
futurodi['x'] = futurodi['f(3,2:t)'] - futurodi['r(1:t)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $x=3$, a regressão é


$$
h(3,2:t+1) - r(1:t) = \alpha + \beta \left[f(3,2:t) - r(1:t)\right] + u_2(t+1)
$$

In [16]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.011
Model:                            OLS   Adj. R-squared:                  0.003
Method:                 Least Squares   F-statistic:                     1.388
Date:                Fri, 21 Oct 2022   Prob (F-statistic):              0.241
Time:                        19:42:02   Log-Likelihood:                 368.50
No. Observations:                 125   AIC:                            -733.0
Df Residuals:                     123   BIC:                            -727.3
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0248      0.003      8.880      0.0

In [17]:
futurodi['y'] = futurodi['h(4,3:t+1)'] - futurodi['r(1:t)']
futurodi['x'] = futurodi['f(4,3:t)'] - futurodi['r(1:t)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $x=4$, a regressão é


$$
h(4,3:t+1) - r(1:t) = \alpha + \beta \left[f(4,3:t) - r(1:t)\right] + u_2(t+1)
$$

In [18]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.013
Model:                            OLS   Adj. R-squared:                  0.005
Method:                 Least Squares   F-statistic:                     1.601
Date:                Fri, 21 Oct 2022   Prob (F-statistic):              0.208
Time:                        19:42:02   Log-Likelihood:                 310.14
No. Observations:                 125   AIC:                            -616.3
Df Residuals:                     123   BIC:                            -610.6
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0282      0.006      4.878      0.0

In [19]:
futurodi['y'] = futurodi['h(5,4:t+1)'] - futurodi['r(1:t)']
futurodi['x'] = futurodi['f(5,4:t)'] - futurodi['r(1:t)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $x=5$, a regressão é


$$
h(5,4:t+1) - r(1:t) = \alpha + \beta \left[f(5,4:t) - r(1:t)\right] + u_2(t+1)
$$

In [20]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.131
Model:                            OLS   Adj. R-squared:                  0.124
Method:                 Least Squares   F-statistic:                     18.53
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           3.38e-05
Time:                        19:42:03   Log-Likelihood:                 273.67
No. Observations:                 125   AIC:                            -543.3
Df Residuals:                     123   BIC:                            -537.7
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept      0.0155      0.008      1.846      0.0

Para replicar o primeiro modelo da Tabela 3, basta alterar a variável $h(x,x-1:t+1)$ por $r(1:t+x-1)$, isso é

$$
r(1:t+x-1) - r(1:t) = \alpha + \beta \left[f(x,x-1:t) - r(1:t)\right] + u_2(t+1)
$$

onde novamente, $x \in \{2,3,4,5\}$

In [21]:
futurodi['y'] = futurodi['r(1:t+1)'] - futurodi['r(1:t)']
futurodi['x'] = futurodi['f(2,1:t)'] - futurodi['r(1:t)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $x=2$, a regressão é


$$
r(1:t+1) - r(1:t) = \alpha + \beta \left[f(2,1:t) - r(1:t)\right] + u_2(t+1)
$$

In [22]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.829
Model:                            OLS   Adj. R-squared:                  0.827
Method:                 Least Squares   F-statistic:                     595.4
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           5.74e-49
Time:                        19:42:03   Log-Likelihood:                 463.51
No. Observations:                 125   AIC:                            -923.0
Df Residuals:                     123   BIC:                            -917.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0120      0.001    -11.969      0.0

In [23]:
futurodi['y'] = futurodi['r(1:t+2)'] - futurodi['r(1:t)']
futurodi['x'] = futurodi['f(3,2:t)'] - futurodi['r(1:t)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $x=3$, a regressão é


$$
r(1:t+2) - r(1:t) = \alpha + \beta \left[f(3,2:t) - r(1:t)\right] + u_2(t+1)
$$

In [24]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.666
Model:                            OLS   Adj. R-squared:                  0.663
Method:                 Least Squares   F-statistic:                     245.1
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           4.64e-31
Time:                        19:42:03   Log-Likelihood:                 375.52
No. Observations:                 125   AIC:                            -747.0
Df Residuals:                     123   BIC:                            -741.4
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0221      0.003     -8.385      0.0

In [25]:
futurodi['y'] = futurodi['r(1:t+3)'] - futurodi['r(1:t)']
futurodi['x'] = futurodi['f(4,3:t)'] - futurodi['r(1:t)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $x=4$, a regressão é


$$
r(1:t+3) - r(1:t) = \alpha + \beta \left[f(4,3:t) - r(1:t)\right] + u_2(t+1)
$$

In [26]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.373
Model:                            OLS   Adj. R-squared:                  0.368
Method:                 Least Squares   F-statistic:                     73.07
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           4.08e-14
Time:                        19:42:04   Log-Likelihood:                 328.28
No. Observations:                 125   AIC:                            -652.6
Df Residuals:                     123   BIC:                            -646.9
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0218      0.005     -4.368      0.0

In [27]:
futurodi['y'] = futurodi['r(1:t+4)'] - futurodi['r(1:t)']
futurodi['x'] = futurodi['f(5,4:t)'] - futurodi['r(1:t)']
result = sm.ols(formula="y ~ x", data=futurodi).fit()

Para $x=5$, a regressão é


$$
r(1:t+4) - r(1:t) = \alpha + \beta \left[f(5,4:t) - r(1:t)\right] + u_2(t+1)
$$

In [28]:
print(result.summary())

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.138
Model:                            OLS   Adj. R-squared:                  0.131
Method:                 Least Squares   F-statistic:                     19.67
Date:                Fri, 21 Oct 2022   Prob (F-statistic):           2.01e-05
Time:                        19:42:04   Log-Likelihood:                 304.58
No. Observations:                 125   AIC:                            -605.2
Df Residuals:                     123   BIC:                            -599.5
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
Intercept     -0.0086      0.007     -1.313      0.1